In this assignment you will be asked to extend the work by Gatti et al by checking whether form-meaning mappings learned on a different yet related language to that considered in the original study still capture the perceived valence of pseudowords. To do this you will be asked to engage with several different resources and adapt the pipeline following the instructions. Along the way, you will be asked to answer a few questions.

You need to submit the complete notebook in .ipynb format, with intermediate outputs visible. The notebook should be named as follows:

CL2025_groupN_assignment.ipynb

where N is the group number. Submissions in the wrong format or with names not adhering to the guidelines will not be evaluated.

Indicate group members' names, student numbers, and contributions below:
- 1.
- 2.
- 3.
- 4.
- 5.

In [1]:
# the code has been tested using the psycho-embeddings library to extract representations from LLMs. You can also use other libraries,
# as long as you make sure that you are producing the correct output.
!git clone https://github.com/MilaNLProc/psycho-embeddings.git
%cd psycho-embeddings
!pip install datasets


fatal: destination path 'psycho-embeddings' already exists and is not an empty directory.
/Users/bramdewaal/Desktop/Uni/VSC/CL/Group Assignment/psycho-embeddings


/Users/bramdewaal/Library/Python/3.11/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


DEPRECATION: Loading egg at /Users/bramdewaal/anaconda3/lib/python3.11/site-packages/colorcorrect-0.9.1-py3.11-macosx-11.1-arm64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [26]:
%pip install nltk
%pip install fasttext
%pip install psycho_embeddings 
%pip install pyreadr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
# the solution to the assignment has been obtained using these packages.
# you're free to use other packages though: consider this as an indication, not a prescription.
import nltk
import numpy as np
import pandas as pd
import fasttext as ft
import pickle as pkl
import fasttext.util
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
from psycho_embeddings import ContextualizedEmbedder

**Task 1** (*10 points available, see breakdown per task below*)

You should replicate the main design in the paper *Valence without meaning* by Gatti and colleagues (2024), using estimates collected for Dutch word valence to train linear regression models and apply them to predict the valence of English pseudowords from Gatti and colleagues.

In detail, to train your regression models, you should use the dataset by Speed and Brysbaert (2024) containing crowd-sourced valence ratings (use the metadata to identify the relevant columns) collected for approximately 24,000 Dutch words. See the paper *Ratings of valence, arousal, happiness, anger, fear, sadness, disgust, and surprise for 24,000 Dutch words* by Speed and Brysbaert (2024).

You should train a letter unigram model and a bigram model. Each model should be trained on Dutch words only.

Pay attention to one issue though: pseudowords created for English may be valid words in Dutch: therefore, you should first filter the list of pseudowords against a large store of Dutch words. To do so, use the words in the Dutch prevalence lexicon available in this OSF repository: https://osf.io/9zymw/. Essentially, you need to exclude any pseudoword that happens to be a word for which a prevalence estimate is available, whatever the prevalence is.

Each code block indicates how many points are available and how they are attributed.

In [29]:
import pyreadr

# Load the RData file
result = pyreadr.read_r("/Users/bramdewaal/Desktop/Uni/VSC/CL/Group Assignment/data/data_pseudovalence.Rdata")

# Extract the first data frame (assuming it's the one you need)
gatti_df = list(result.values())[0]

# Preview the first 5 rows
print(gatti_df.head())


             Valence  predicted_val  predicted_valL  predicted_valL_BI  \
rownames                                                                 
aardvark        6.26       6.392012        4.920180           6.410768   
abalone         5.30       4.756492        5.284912           5.115389   
abandon         2.84       4.260055        5.001226           5.479860   
abandonment     2.63       4.196807        5.022504           5.334364   
abbey           5.85       6.123953        5.147159           5.162931   

             predicted_valDIM  predicted_valL_DIM  predicted_valBI  \
rownames                                                             
aardvark             5.772722            5.774341         6.410768   
abalone              4.728264            4.858120         5.115389   
abandon              3.978241            3.987623         5.479860   
abandonment          3.833330            3.828077         5.334364   
abbey                6.064834            6.094675         5.1

In [31]:
# read in the pseudowords from Gatti and colleagues, 
# as well as the valence ratings for 24,000 Dutch words from Speed and Brysbaert (2024)
# show the first 5 lines of each dataset.
# 1 point for identifying the correct files and correctly loading their content

import pyreadr
import pandas as pd

# Load the Gatti et al. (2024) pseudoword valence dataset from .Rdata
gatti_result = pyreadr.read_r("/Users/bramdewaal/Desktop/Uni/VSC/CL/Group Assignment/data/data_pseudovalence.Rdata")
gatti_df = list(gatti_result.values())[0]

print("Gatti et al. pseudoword valence dataset:")
print(gatti_df.head())

# Load the Speed & Brysbaert (2024) Dutch word valence dataset from .csv
speed_df = pd.read_csv("/Users/bramdewaal/Desktop/Uni/VSC/CL/Group Assignment/data/prevalence_netherlands.csv")

print("\nSpeed & Brysbaert Dutch valence dataset:")
print(speed_df.head())


Gatti et al. pseudoword valence dataset:
             Valence  predicted_val  predicted_valL  predicted_valL_BI  \
rownames                                                                 
aardvark        6.26       6.392012        4.920180           6.410768   
abalone         5.30       4.756492        5.284912           5.115389   
abandon         2.84       4.260055        5.001226           5.479860   
abandonment     2.63       4.196807        5.022504           5.334364   
abbey           5.85       6.123953        5.147159           5.162931   

             predicted_valDIM  predicted_valL_DIM  predicted_valBI  \
rownames                                                             
aardvark             5.772722            5.774341         6.410768   
abalone              4.728264            4.858120         5.115389   
abandon              3.978241            3.987623         5.479860   
abandonment          3.833330            3.828077         5.334364   
abbey               

In [5]:
# filter out pseudowords that happen to be valid Dutch words (mind case folding!)
# show the set of pseudowords filtered out.
# 1 point for applying the correct filtering

In [6]:
# encode Dutch words and pseudowords from Gatti et al as uni- and bi-gram vectors
# show the uni-gram and bi-gram encoding of the pseudoword ampgrair
# 2 points for correctly encoding the target strings as uni- and bi-gram vectors

In [7]:
# use word valence estimates from Speed and Brysbaert (2024) to train
# - a uni-gram model
# - a bi-gram model
# 2 points for correctly trained models

In [8]:
# apply trained models to predict the valence of pseudowords from Gatti et al (2024).
# Then apply the same models back onto the training set to see how well they predict the valence of words in Speed and Brysbaert (2024).
# 2 points for correctly applied models

In [9]:
# compute the Spearman correlation coefficients between true valence and predicted valence under both uni- and bi-gram models for
# - words from Speed and Brysbaert (2024)
# - pseudowords from Gatti and colleagues (2024)
# show both correlation coefficients.
# 2 points for the correct Spearman correlation coefficients (rounded to the third decimal place)

**Task 2** (*8 points available, see breakdown below*)

Again following Gatti and colleagues, you should encode the target strings (pseudowords and Dutch words from Speed and Brysbaert) as fastText embeddings, train a multiple regression model on Dutch words and apply it to the pseudowords in Gatti et al. You should finally report the Spearman correlation coefficient between observed and predicted valence for both words and pseudowords.

You should use the pre-trained fastText model for Dutch, available at this page: https://fasttext.cc/docs/en/crawl-vectors.html

Finally, you should answer two questions about the fastText model (see below).

In [10]:
# load the fastText model
# 1 point for correctly loading the appropriate fastText model

What is the dimensionality of the pre-trained Dutch fastText embeddings? (*1 point for the correct answer*)

What minimum and maximum n-gram size was specified for training this fastText model? (*1 point for the correct answer*)

In [11]:
# encode Dutch words and pseudowords as fastText embeddings
# show the first 20 values of the embedding of the word 'speelplaats' and of the pseudoword 'danchunk'
# 2 points for correctly encoding words and pseudowords with fastText

In [12]:
# train regression model on word valence
# 1 point for correctly training the regression model

In [13]:
# apply the trained model to predict the valence of pseudowords from Gatti et al (2024).
# Then apply the same model back onto the training set to see how well it predicts the valence of words in Speed and Brysbaert (2024).
# 1 point for correctly applied model

In [14]:
# compute the Spearman correlation coefficients between true valence and predicted valence for
# - words from Speed and Brysbaert (2024)
# - pseudowords from Gatti and colleagues (2024)
# show the correlation coefficient.
# 1 point for the correct Spearman correlation coefficients (rounded to the third decimal place)

**Task 3** (*6 points available, see breakdown below*)

Now you are asked to extend the work by Gatti et al by also considering the representations learned by a transformer-based models, in detail *RobBERT v2* (https://huggingface.co/pdelobelle/robbert-v2-dutch-base). You should follow the same pipeline as for the previous models, encoding both Dutch words from Speed and Brysbaert (2024) and the pseudowords from Gatti et al using the embedding of each string at layer 0, before positional information is factored in. If a string consists of multiple tokens, average the embeddings of all tokens to produce the embedding of the whole string. Then train a multiple regression model on the valence of Dutch words, apply it to the pseudowords, and compute the Spearman correlation between observed and predicted ratings.

Use the HuggingFace model card for RobBERT v2 to check how to access it.

I recommend saving the embeddings to file once you have generated them and you know they are correct: embedding thousands of strings takes some time, and you don't want to have to do it again. For the same reason, develop your code by considering only a small fractions of the words and pseudowords, in order to quickly see if something is wrong. Only when you are positive it works, embed all strings.

In [15]:
# load and instantiate the right model
# 1 point for loading the right model

In [16]:
# encode the words and pseudowords using RobBERT v2. I've used the free GPU runtime on COLAB to speed things up,
# but in this case you need to batch the words and pseudowords. You can use the function below to create batches
# but you will have to pay attention at how you store embeddings.
# show the first 20 values of the embedding of the word 'miauwen' and of the pseudoword 'lixthless'
# 2 points for correctly encoding words and pseudowords

def chunks(lst, n):

    """Chunks a list into equal chunks containing n elements. Returns a list of lists."""

    chunked = []
    for i in range(0, len(lst), n):
        chunked.append(lst[i:i + n])
    return chunked


In [17]:
# train regression model on word valence estimates from Speed and Brysbaert (2024)
# 1 point for correctly training the regression model

In [18]:
# apply the trained model to predict the valence of pseudowords from Gatti et al (2024).
# Then apply the same model back onto the training set to see how well it predicts the valence of words in Speed and Brysbaert (2024).
# 1 point for correctly applied model

In [19]:
# compute the Spearman correlation coefficients between true valence and predicted valence for
# - words from Speed and Brysbaert (2024)
# - pseudowords from Gatti and colleagues (2024)
# show the correlation coefficient
# 1 point for the correct Spearman correlation coefficients (rounded to the third decimal place)

**Task 4** (*16 points available, 4 for each question*)

Answer the following questions.

**4a.** Describe the performance of each featurization, comparing
- the performance of a same model between the training and test set
- the performance of different models on the training set
- the performance of different models on the test set

(*4 points available, max 150 words*)

*type your answer here*

**4b.** Compare the correlations you found when training uni-gram, bi-gram, and fastText models on Dutch words and the correlations of similar models trained on English data as reported by Gatti and colleagues; summarize the most important similarities and differences.

(*4 points available, max 150 words*)

*type your answer here*

**4c.** Do you think the performance of the fastText featurization would change if you were to use different n-grams? Would you make them smaller or larger? Justify your answer.

(*4 points available, max 150 words*)

*type your answer here*

**4d.** Do you think that training the same models on uni-grams, bi-grams, fastText and transformer-based embeddings but using valence ratings for Finnish (a language which uses the same alphabet as English but is not a IndoEuropean language) words would yield a similar pattern of results? Justify your answer.

(*4 points available, max 150 words*)

*type your answer here*

**Task 5** (*3 points available*)

Compute the average Levenshtein Distance (aLD) between each pseudoword and the 20 words at the smallest edit distance from it. Consider the set of words you used to filter out pseudowords that happen to be valid Dutch words (the file is available in this OSF repository: https://osf.io/9zymw/) to retrieve the 20 words at the smallest edit distance.

In [20]:
# compute the average Levenshtein distance from each pseudoword to the words used to filter out pseudowords.
# Show the aLD estimate for the pseudowords 'nedukes', 'pewbin', and 'vibcines'
# 3 points for correctly computing aLD for pseudowords

**Task 6** (*3 points available*)

For each pseudoword, record the number of tokens in which RobBERT v2 encodes it.

In [21]:
# record the number of tokens in which RobBERT divides each pseudoword
# show the number of tokens for the pseudowords 'yuxwas', 'skibfy', and 'errords'
# 3 points for correctly mapping pseudowords to number of tokens

**Task 7** (*5 points available, see breakdown below*)

Compute the residuals of the predicted valence under the four regressors trained and applied in tasks 2 to 4. Then, correlate the residuals from all four models with aLD. Finally, correlate the residuals from the RobBERT v2 model with the number of tokens in which each pseudoword is split. Use the Pearson's correlation coefficient.

In [22]:
# compute the residuals from all four regression models fitted before
# 1 point available for correctly computing residuals

In [23]:
# compute the Pearson's correlation between residuals and average LD for all models,
# as well as the correlation between RobBERT v2 residuals and the number of tokens in which each pseudoword
#    is encoded by the RobBERT v2 model.
# show all correlation coefficients
# 4 points for the correct correlation coefficients

**Task 8** What is the relation between the errors each model made and aLD? what about the number of tokens (limited to the RobBERT v2 model)?

(*4 points available, max 150 words*)

*testo in corsivo*